<a href="https://colab.research.google.com/github/pgssanjana/micro_projects/blob/main/face_recognition_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle datasets list


Saving kaggle.json to kaggle.json


In [2]:
!kaggle datasets download -d pramod722445/face-recognition-30


 99% 638M/641M [00:04<00:00, 118MB/s]
100% 641M/641M [00:04<00:00, 142MB/s]


In [4]:
import zipfile
zf = "/content/face-recognition-30.zip"
target_dir = "/content/input/face-recognition-30/dataset"
zfile = zipfile.ZipFile(zf)
zfile.extractall(target_dir)

In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib as mpl

# style your matplotlib
mpl.style.use("seaborn-darkgrid")
# run this block



In [6]:
from tqdm import tqdm

In [7]:
files=os.listdir("/content/input/face-recognition-30/dataset/dataset/")
files

['Jim_Parsons',
 'Matthew_Perry',
 'dhoni',
 'virat_kohli',
 'mohamed_ali',
 'aishwarya_rai',
 'random_person',
 'Simon_Helberg',
 'ronaldo',
 'Courteney_Cox',
 'Matt_LeBlanc',
 'angelina_jolie',
 'brad_pitt',
 'sylvester_stallone',
 'Kunal_Nayya',
 'Jennifer_Aniston',
 'scarlett_johansson',
 'pewdiepie',
 'Lisa_Kudrow',
 'arnold_schwarzenegger',
 'Pankaj_Tripathi',
 'hardik_pandya',
 'David_Schwimmer',
 'messi',
 'Sachin_Tendulka',
 'ROHIT_SHARMA',
 'Johnny_Galeck',
 'manoj_bajpayee',
 'suresh_raina',
 'bhuvan_bam']

In [8]:
image_array=[]  # it's a list later i will convert it to array
label_array=[]
path="/content/input/face-recognition-30/dataset/dataset/"
# loop through each sub-folder in train
for i in range(len(files)):
    # files in sub-folder
    file_sub=os.listdir(path+files[i])

    for k in tqdm(range(len(file_sub))):
        try:
            img=cv2.imread(path+files[i]+"/"+file_sub[k])
            img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            img=cv2.resize(img,(96,96))
            image_array.append(img)
            label_array.append(i)
        except:
            pass

100%|██████████| 382/382 [00:00<00:00, 864.12it/s]


In [9]:
import gc
gc.collect()

88

In [10]:
image_array=np.array(image_array)/255.0
label_array=np.array(label_array)


In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(image_array,label_array,test_size=0.15)

In [12]:
from keras import layers,callbacks,utils,applications,optimizers
from keras.models import Sequential,Model,load_model

In [13]:
len(files)


30

In [14]:
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator
# from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input

In [15]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [16]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

In [18]:

model=Sequential()
# I will use MobileNetV2 as an pretrained model 
pretrained_model=MobileNetV2(input_shape=(96,96,3),include_top=False,
                                         weights="imagenet")
model.add(pretrained_model)
model.add(layers.GlobalAveragePooling2D())
# add dropout to increase accuracy by not overfitting
model.add(layers.Dropout(0.3))
# add dense layer as final output
model.add(layers.Dense(1))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_96 (Functi  (None, 3, 3, 1280)       2257984   
 onal)                                                           
                                                                 
 global_average_pooling2d_1   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_1 (Dropout)         (None, 1280)              0         
                                                                 
 dense_1 (Dense)             (None, 1)                 1281      
                                                                 
Total params: 2,259,265
Trainable params: 2,225,153
Non-trainable params: 34,112
_________________________________________________________________


In [19]:
model.compile(optimizer="adam",loss="mean_squared_error",metrics=["mae"])



In [20]:

ckp_path="trained_model/model"
model_checkpoint=tf.keras.callbacks.ModelCheckpoint(filepath=ckp_path,
                                                   monitor="val_mae",
                                                   mode="auto",
                                                   save_best_only=True,
                                                   save_weights_only=True)


In [21]:
# create a lr reducer which decrease learning rate when accuarcy does not increase
reduce_lr=tf.keras.callbacks.ReduceLROnPlateau(factor=0.9,monitor="val_mae",
                                             mode="auto",cooldown=0,
                                             patience=5,verbose=1,min_lr=1e-6)
# patience : wait till 5 epoch
# verbose : show accuracy every 1 epoch
# min_lr=minimum learning rate

In [ ]:
# import tensorflow as tf
# my_callbacks = [
#     tf.keras.callbacks.EarlyStopping(patience=2),
#     tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5',
#                               monitor='val_loss',
#                              verbose=1, 
#                              save_best_only=True),
#     tf.keras.callbacks.TensorBoard(log_dir='./logs'),
# ]

In [22]:
EPOCHS=150
BATCH_SIZE=64

history=model.fit(X_train,
                 Y_train,
                 validation_data=(X_test,Y_test),
                 batch_size=BATCH_SIZE,
                 epochs=EPOCHS,
                 callbacks=[model_checkpoint,reduce_lr],
                  steps_per_epoch=128
                  

                 )

Epoch 1/150
128/128 [==============================] - 32s 113ms/step - loss: 51.1929 - mae: 5.3230 - val_loss: 180.3858 - val_mae: 11.6756 - lr: 0.0010
Epoch 2/150
128/128 [==============================] - 12s 92ms/step - loss: 22.9158 - mae: 3.4151 - val_loss: 296.6952 - val_mae: 15.7995 - lr: 0.0010
Epoch 3/150
128/128 [==============================] - 12s 91ms/step - loss: 18.8766 - mae: 2.9621 - val_loss: 504.3993 - val_mae: 20.5910 - lr: 0.0010
Epoch 4/150
128/128 [==============================] - 12s 91ms/step - loss: 16.3437 - mae: 2.7420 - val_loss: 195.2178 - val_mae: 12.4348 - lr: 0.0010
Epoch 5/150
128/128 [==============================] - 11s 87ms/step - loss: 13.8546 - mae: 2.4659 - val_loss: 228.6660 - val_mae: 13.3170 - lr: 0.0010
Epoch 6/150
128/128 [==============================] - ETA: 0s - loss: 10.0804 - mae: 2.0625
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0009000000427477062.
128/128 [==============================] - 12s 91ms/step - loss: 10.08

In [23]:
prediction_val=model.predict(X_test,batch_size=64)
prediction_val[:20]

array([[12.759879  ],
       [15.578155  ],
       [13.156936  ],
       [19.89915   ],
       [ 9.682581  ],
       [18.785875  ],
       [17.665398  ],
       [ 6.8373237 ],
       [19.410976  ],
       [11.27731   ],
       [ 6.2635107 ],
       [ 0.06841336],
       [ 0.85455227],
       [15.999194  ],
       [ 5.2215505 ],
       [27.208534  ],
       [ 5.0695086 ],
       [17.141598  ],
       [ 9.377102  ],
       [ 2.7974699 ]], dtype=float32)

In [25]:
model.load_weights(ckp_path)

In [26]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpc9nhhmyp/assets


INFO:tensorflow:Assets written to: /tmp/tmpc9nhhmyp/assets
